In [ ]:
!pip install -q /kaggle/input/iterative-stratification/iterative-stratification-master
!pip install git+https://github.com/qubvel/segmentation_models.pytorch -q

In [ ]:
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import patches, text, patheffects
from pylab import rcParams
import seaborn as sns
from tqdm.auto import tqdm

In [ ]:
cls = ["Negative for Pneumonia", "Typical Appearance", "Indeterminate Appearance", "Atypical Appearance"]

masks_path = '../input/maskcovid/masks/'
boxes = pd.read_csv("../input/siim-covid19-detection-512/train.csv")
boxes["image_id"] = boxes["id_image"]
train = pd.read_csv("../input/siimcovid19-512-jpg-image-dataset/train.csv")
train = pd.merge(train, boxes[["image_id", "width", "height"]], on="image_id", how="left")
train["file_path"] = "../input/siimcovid19-512-jpg-image-dataset/train/" + train["image_id"] + ".jpg"
train["class_name"] = np.argmax(train[cls].values, axis=1)
# train["n_box"] = np.nan
# train["n_box"] = train["label"].apply(lambda x: 0 if x.split(" ")[0] == "none" else len(x.split(" "))//6)
# train_n1 = train[train["class_name"]!=0].copy()
# train_n1 = train[train["n_box"]>0]
train["mask_path"] = masks_path + train["image_id"] + "_mask" + ".jpg"
train.reset_index(inplace=True)

display(train.head())

In [ ]:
print(train.class_name.value_counts())
train['class_name'].hist()

In [ ]:
class CFG:
    
    apex=True
    debug=False
    train=True
    mixup = False
    print_freq=50
    target_size=len(cls)
    target_col=cls
    trn_fold=[0, 1, 2, 3, 4]
    batch_size=28
    n_fold=5
    
    model_name='efficientnet-b0'
    
    weights=[None]*n_fold
    
#     weights=["../input/tempmodels/efficientnet-b5_fold0_best_score.pth",
#              "../input/segmodelaux/efficientnet-b3_fold1_best_score.pth",
#              "../input/segmodelaux/efficientnet-b3_fold2_best_score.pth",
#              "../input/segmodelaux/efficientnet-b3_fold3_best_score.pth",
#              "../input/segmodelaux/efficientnet-b3_fold4_best_score.pth"]

    
    
    size=512
    
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=6 # CosineAnnealingLR
    #T_0=6 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=3e-5
    weight_decay=0
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    num_workers=4
    freeze=False
    
if CFG.debug:
    
    CFG.epochs = 1
    train = train.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)

In [ ]:
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
import torchvision.transforms as T
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import segmentation_models_pytorch as smp
import torchvision

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

from torch.cuda.amp import autocast, GradScaler

import warnings 
warnings.filterwarnings('ignore')

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Python        : ' + sys.version.split('\n')[0])
print('Numpy         : ' + np.__version__)
print('Pandas        : ' + pd.__version__)
print('PyTorch       : ' + torch.__version__)
print('Albumentations: ' + A.__version__)
print('Timm          : ' + timm.__version__)

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = average_precision_score(y_true, y_pred)
    return score * (2/3)


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

In [ ]:
y = train[cls].values
X = train['image_id'].values

train['fold'] = np.nan

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
mskf = MultilabelStratifiedKFold(n_splits=CFG.n_fold, random_state=CFG.seed, shuffle=True)
for i, (_, test_index) in enumerate(mskf.split(X, y)):
    train.iloc[test_index, -1] = i
    
train['fold'] = train['fold'].astype('int')
display(train.groupby(['fold', 'class_name']).size())

In [ ]:
class TrainDataset(Dataset):
    
    def __init__(self, df, transforms=None, train=True):
        
        super().__init__()
        self.df = df
        self.transforms = transforms
        self.train = train
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
            
        row = self.df.iloc[idx]
        img = cv2.imread(row.file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        label = row[CFG.target_col].astype('int').values
        
        if self.train:
            mask = np.ones([CFG.size,CFG.size])
            if row.mask_path.split("/")[-1] in os.listdir(masks_path):
                mask_c = cv2.imread(row.mask_path)
                mask_c = cv2.cvtColor(mask_c, cv2.COLOR_BGR2GRAY)
                mask_c = cv2.resize(mask_c, mask.shape[0:2], interpolation = cv2.INTER_AREA)
                mask = (mask_c/200).astype(int)
                mask *= row.class_name
                

        if self.transforms is not None:
            if self.train:
                transformed = self.transforms[0](image=img, mask=mask)
                img = transformed["image"]
                mask = transformed["mask"]
                img = self.transforms[1](image=img)["image"]
                mask = torchvision.transforms.functional.to_tensor(mask)
            else:
                img = self.transforms(image=img)["image"]
        else:
            img = torchvision.transforms.functional.to_tensor(img).float()
            mask = torchvision.transforms.functional.to_tensor(mask).float()
            
        if row.mask_path.split("/")[-1] not in os.listdir(masks_path):
            mask = torch.ones([CFG.size,CFG.size]) * -100
        
        label = torch.argmax(torch.as_tensor(label))
        
        if self.train:
            return img, label, mask.int().squeeze()
        
        return img, label

In [ ]:
def get_transforms(): 
    
    transforms_train_all = A.Compose([
        #A.VerticalFlip(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.RandomCrop(CFG.size,CFG.size),
        A.ShiftScaleRotate(shift_limit=0, scale_limit=0.1, rotate_limit=45, border_mode=0, p=0.75),
    ])
    
    transforms_train_image = A.Compose([
        A.RandomBrightness(limit=0.1, p=0.75),
        #A.HueSaturationValue(p=0.5, hue_shift_limit=0, sat_shift_limit=(0.1,0.2), val_shift_limit=0),
        A.Cutout(max_h_size=int(CFG.size * 0.12), max_w_size=int(CFG.size * 0.12), num_holes=3, p=0.5),
        A.Normalize(),
        ToTensorV2()
    ])
    
    transforms_val = A.Compose([
        A.Resize(CFG.size, CFG.size),
        #A.HueSaturationValue(p=0.5, hue_shift_limit=0, sat_shift_limit=(0.1,0.2), val_shift_limit=0),
        A.Normalize(),
        ToTensorV2()
    ])
    
    return (transforms_train_all,transforms_train_image), transforms_val

def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [ ]:
transforms_train, transforms_valid = get_transforms()
ds = TrainDataset(train, transforms_train)
ds[1][2].shape

In [ ]:
def draw_outline(obj):
    obj.set_path_effects([patheffects.Stroke(linewidth=4, foreground='black'), patheffects.Normal()])

def draw_box(ax, bb):
    patch = ax.add_patch(patches.Rectangle((bb[0],bb[1]), bb[2], bb[3], fill=False, edgecolor='red', lw=2))
    draw_outline(patch)

rcParams['figure.figsize'] = 15,5
for i in range(2):
    f, axarr = plt.subplots(1,5)
    for p in range(5):
        idx = i*5 + p
        img, label, mask = ds[idx]
        axarr[p].imshow(img.transpose(0,1).transpose(1,2).squeeze())
        axarr[p].imshow(mask, interpolation='none', alpha=0.25)
        

In [ ]:
def get_Unet(encoder_name, n_classes, pretrained=True):
    
    aux_params=dict(
    pooling='avg',             # one of 'avg', 'max'
    dropout=0.5,               # dropout ratio, default is None
    activation=None,      # activation function, default is None
    classes=n_classes,                 # define number of output labels
    )
    
    if pretrained:
        weights='imagenet'
    else:
        weights=None
        
    model =  smp.UnetPlusPlus(
                 encoder_name=encoder_name,
                 encoder_weights=weights,
                 in_channels=3,
                 classes=n_classes, 
                aux_params=aux_params)
    return model

class CustomModel(nn.Module):
    
    def __init__(self, cfg, pretrained=False):
        super().__init__()
        self.cfg = cfg
        self.unet = get_Unet(self.cfg.model_name, self.cfg.target_size, pretrained=pretrained)
        
    def forward(self, x, train=True):
        
        if train:
            return self.unet(x)
        
        e = self.unet.encoder(x)[-1]
        return self.unet.classification_head(e)

In [ ]:
model = CustomModel(CFG, True)
x = torch.stack([ds[i][0] for i in range(2)])
y = model(x)
y[1]

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(train_loader, model, criterion,  optimizer, epoch, scheduler, device):
    if CFG.apex:
        scaler = GradScaler()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, labels, masks) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
            
        images = images.to(device)
        labels = labels.to(device, dtype=torch.int64)
        masks = masks.to(device, dtype=torch.int64)
        batch_size = len(labels)
        
        if CFG.apex:
            with autocast():
                y_preds = model(images)
                loss = criterion(y_preds, (masks, labels))
        else:
            y_preds = model(images)
            loss = criterion(y_preds, (masks, labels))
        
        # record loss
        #l_u = nn.BCEWithLogitsLoss()(y_preds, nn.functional.one_hot(labels,4).float())
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            if CFG.apex:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   lr=scheduler.get_lr()[0],
                   ))
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device, dtype=torch.int64)
        batch_size = len(labels)
        # compute loss
        with torch.no_grad():
            y_preds = model(images, train=False)
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        act = nn.Softmax(1)
        preds.append(act(y_preds).to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [ ]:
# ====================================================
# Train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_col].values
    
    transform_train, transform_valid = get_transforms()

    train_dataset = TrainDataset(train_folds, 
                                 transforms=transform_train,
                                 train=True)
    
    valid_dataset = TrainDataset(valid_folds, 
                                 transforms=transform_valid,
                                 train=False)

    train_loader = DataLoader(train_dataset, 
                              batch_size=CFG.batch_size, 
                              shuffle=True, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    
    valid_loader = DataLoader(valid_dataset, 
                              batch_size=CFG.batch_size * 2, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    
    # ====================================================
    # scheduler 
    # ====================================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, 
                                          verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, 
                                                    last_epoch=-1)
        return scheduler

    # ====================================================
    # Loss Function , dtype=torch.int64
    # ====================================================
    
#     class custom_loss(nn.Module):
        
#         def __init__(self, bb_weight=1e-2):
#             super(custom_loss, self).__init__()
#             self.bb_weight = bb_weight

#         def forward(self, y_preds, labels):        

#             y_p, b_p = y_preds
#             y_l, b_l = labels       

#             return nn.BCEWithLogitsLoss()(y_p, y_l) + self.bb_weight * nn.MSELoss(reduction="mean")(b_p, b_l)

    
        
    class CompositeLoss(nn.Module):
        #smp.losses.DiceLoss(mode='multiclass', smooth=0.05)
        def __init__(self, losses=[smp.losses.SoftCrossEntropyLoss(smooth_factor=0.15, ignore_index=-100), 
                        nn.CrossEntropyLoss()], weights=[1.0, 1.0]):
            super().__init__()
            assert len(losses) == len(weights)
            self.losses = losses
            self.weights = weights

        def forward(self, approxs, targets):
            total_loss = 0.0
            for i in range(len(self.losses)):
                total_loss += self.weights[i] * self.losses[i](
                    approxs[i], targets[i])
            return total_loss
                    
    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, pretrained=True)
        
    if CFG.weights[fold] is not None:
        print(f"Loading pre-trained weights for fold-{fold}.......")
        model.load_state_dict(torch.load(CFG.weights[fold], map_location=device)['model'])
    model.to(device)

    optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = get_scheduler(optimizer)

    # ====================================================
    # loop
    # ====================================================
    train_criterion = CompositeLoss(weights=[0.2, 1.0])
    val_criterion = nn.CrossEntropyLoss()
    
    best_score = 0.
    best_loss = np.inf
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()
        
        # train
        avg_loss = train_fn(train_loader, model, train_criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, val_criterion, device)
        
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()

        # scoring
        score = get_score(valid_labels, preds)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')

        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_score.pth')
        
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            LOGGER.info(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_loss.pth')
    
    valid_folds[["Pred_"+s for s in CFG.target_col]] = torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_loss.pth', 
                                      map_location=torch.device('cpu'))['preds']

    return valid_folds

In [ ]:
# ====================================================
# main
# ====================================================
def main():

    """
    Prepare: 1.train 
    """

    def get_result(result_df):
        preds = result_df[["Pred_"+s for s in CFG.target_col]].values
        labels = result_df[CFG.target_col].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}')
    
    if CFG.train:
        # train 
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)

In [ ]:
if __name__ == '__main__':
    main()